In [27]:
import re
import json
import pickle
import os
import sys
import requests
import logging
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
from collections import Counter
from datasets import load_dataset
import pandas as pd
from ipywidgets import interact, IntSlider
from process_tiny_stories_data import load_tinystories_validation_prompts, load_tinystories_tokens
from typing import Literal


pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

logging.basicConfig(format='(%(levelname)s) %(asctime)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')
sys.path.append('../')  # Add the parent directory to the system path

import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import custom_forward, AutoEncoderConfig, evaluate_autoencoder_reconstruction, get_encoder_feature_frequencies, load_encoder, get_acts
import utils.haystack_utils as haystack_utils
from utils.plotting_utils import line
from sparse_coding.spacy_tag import make_spacy_feature_df


%reload_ext autoreload
%autoreload 2

In [28]:
import subprocess
subprocess.run(['python', '-m', 'spacy', 'download', 'en_core_web_trf'])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 11.2 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


CompletedProcess(args=['python', '-m', 'spacy', 'download', 'en_core_web_trf'], returncode=0)

In [30]:
haystack_utils.clean_cache()
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.2"

model_name = "tiny-stories-2L-33M"
print_name = "TinyStories 2L 33M"

model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device,
)

n_prompts = 40
prompts = load_tinystories_validation_prompts(data_path='data/tinystories')[:n_prompts]
tokens = model.to_tokens(prompts)
print(tokens.shape)
df = make_spacy_feature_df(model, tokens)

Loaded pretrained model tiny-stories-2L-33M into HookedTransformer
torch.Size([40, 304])
Starting spacy processing of dataset...


KeyboardInterrupt: 

In [31]:
df[["is_spacy_adj"]]
print(tokens.shape)

torch.Size([40, 304])


In [43]:
from sklearn.metrics import f1_score

# look for encoder features that go on particular spacy attributes
save_name = '18_morning_sun'
encoder, cfg = load_encoder(save_name, model_name, model, save_path='/workspace')

acts = []
for i in range(len(tokens)):
    acts.append(get_acts(tokens[i], model, encoder, cfg))
acts = torch.cat(acts).cpu()

threshold = 0.1
f1_scores = {}
for series_name, series in df.items():
    neuron_binarized = (acts > threshold).T
    for i in range(len(neuron_binarized)):
        f1_scores[(series_name, i)] = f1_score(series, neuron_binarized[i])